<a href="https://colab.research.google.com/github/seanfagan/skyw-credits-parser/blob/main/skyw_credits_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skywind Credits Parser
This script takes in two TSV (Tab Separated Values) files from the Skywind Credits spreadsheet and transforms it into JSON as well as HTML for use on the website. You will need to export two TSVs from the Skywind Credits before you can begin:

- **Roles**: Export a TSV of the "Roles" sheet. The ordering on this sheet matters! Rows on this sheet should be ordered by Team (A-Z), then by YOrderBonus (Z-A), then by Role (A-Z).

- **Contributors**: Export a TSV of the "Contributors" sheet.

In [ ]:
#@title Parse Credits, Step 1 of 2
#@markdown When you run this code, Google will warn you that you are about to run an executable not authored by Google.
#@markdown If you'd like to view the source code that you are about to run, double-click anywhere in this text box.
#@markdown ### Steps
#@markdown 1. Press the Play button to begin the script.
#@markdown 2. You will be prompted to upload the **Roles** tsv file.
#@markdown 3. You will be prompted to upload the **Contributors** tsv file.
#@markdown 4. If parsing is successful, continue on to Step 2.

import csv
import io
import json
import pprint
from google.colab import files

OUTPUT_ROLES = []
OUTPUT_CREDITS = []

# --------------
# FILE 1: ROLES
# -------------
print('\n* Please upload your "Roles.tsv" file now.')
uploaded_role_files = files.upload()

if len(uploaded_role_files) != 1:
    raise Exception('only 1 file man')

for filename, filedata in uploaded_role_files.items():
    print('+====================+')
    print('|      ROLE FILE      |')
    print('+====================+')
    print(f'Parsing role tsv "{filename}"')

    reader = csv.reader(io.StringIO(filedata.decode()), delimiter='\t')
    next(reader)  # skip header
    OUTPUT_ROLES = [row[0] for row in reader]

    print(f'{len(OUTPUT_ROLES)} roles successfully parsed')
    print(OUTPUT_ROLES)

# --------------------
# FILE 2: CONTRIBUTORS
# --------------------
print('\n* Please upload your "Contributors.tsv" file now.')
uploaded_contrib_files = files.upload()

if len(uploaded_contrib_files) != 1:
    raise Exception('only 1 file man')

for filename, filedata in uploaded_contrib_files.items():
    print('+====================+')
    print('|    CONTRIB FILE    |')
    print('+====================+')
    print(f'Parsing contributor tsv "{filename}"')

    # Read the credits file into a list of dictionaries
    reader = csv.DictReader(
        io.StringIO(filedata.decode()), fieldnames=[
            'screenname',
            'proname',
            'use',  # 'S'creenname, 'P'ro-name, [blank] use screenname
            'url',
            'picurl',
            'creditname',     # either screenname or proname, depending on 'use'
            'teams',          # all teams, comma separated
            'roles',          # all roles, comma separated
            'distinctions',   # nepotism
            'yorder',
            'dupecheck',      # says "Unique!" if not a dupe
            'suggestedname',  # freeform text from the application
            'suggestedurl',   # freeform text from the application
            'urlartstation',
            'urlgithub',
            'urlsoundcloud',
            'urllinkedin',
            'urlspotify',
            'urlyoutube',
            'urlother',
        ], delimiter='\t'
    )
    credits = list(reader)

    credits_by_role = {}
    i = 0
    # Iterate through credits
    for c in credits:
        i += 1
        credit_name = c['creditname']
        roles = c['roles'].split(', ')

        # Add person to each role they have
        for role in roles:
            credits_by_role.setdefault(role,[]).append(credit_name)

    # Within each role, order the names alphabetically
    credits_by_role = {k: sorted(v) for k, v in credits_by_role.items()}

    # Order the roles themselves by the order from the role file
    credits_by_role = [(k, credits_by_role[k]) for k in OUTPUT_ROLES if k in credits_by_role]
    OUTPUT_CREDITS = credits_by_role
    print(f'{i} credits successfully processed')
    print(json.dumps(OUTPUT_CREDITS))


In [ ]:
#@title Parse Credits, Step 2 of 2
#@markdown Time to download the fruits of your labor.

DOWNLOAD_FILENAME = 'skyw-credits-YYYYMMDD' #@param {type:"string"}
DOWNLOAD_AS_HTML = False #@param {type:"boolean"}
DOWNLOAD_AS_JSON = False #@param {type:"boolean"}
DOWNLOAD_AS_TXT = True #@param {type:"boolean"}

OUTPUT_HTML = ''
OUTPUT_JSON = ''
OUTPUT_TXT = ''

# HTML templates
HTML_START = f"""
[fusion_builder_container hundred_percent="no" hundred_percent_height="no" hundred_percent_height_scroll="no" hundred_percent_height_center_content="yes" equal_height_columns="no" hide_on_mobile="small-visibility,medium-visibility,large-visibility" background_position="center center" background_repeat="no-repeat" fade="no" background_parallax="none" enable_mobile="no" parallax_speed="0.3" video_aspect_ratio="16:9" video_loop="yes" video_mute="yes" border_style="solid" admin_toggled="no" type="legacy"][fusion_builder_row][fusion_builder_column type="1_1" layout="1_1" spacing="" center_content="no" link="" target="_self" min_height="" hide_on_mobile="small-visibility,medium-visibility,large-visibility" class="" id="" background_color="" background_image="" background_position="left top" background_repeat="no-repeat" hover_type="none" border_color="" border_style="solid" border_position="all" padding_top="" padding_right="" padding_bottom="" padding_left="" margin_top="" margin_bottom="" animation_type="" animation_direction="left" animation_speed="0.3" animation_offset="" last="true" border_sizes_top="0" border_sizes_bottom="0" border_sizes_left="0" border_sizes_right="0" first="true" type="1_1"][fusion_text columns="" column_min_width="" column_spacing="" rule_style="default" rule_size="" rule_color="" content_alignment_medium="" content_alignment_small="" content_alignment="" hide_on_mobile="small-visibility,medium-visibility,large-visibility" sticky_display="normal,sticky" class="" id="" font_size="" fusion_font_family_text_font="" fusion_font_variant_text_font="" line_height="" letter_spacing="" text_color="" animation_type="" animation_direction="left" animation_speed="0.3" animation_offset=""]
<div class="markup-2BOw-j">Skywind is a project born out of love by hundreds of fans. This page represents an attempt to list of all the people who have contributed to Skywind over the years, of whom only some are the active Skywind development team. We are always looking for more volunteers for the project.</div>
Can't find your name, or want to be credited differently? <a href="https://discord.gg/skywind">Contact us on Discord</a> with details of your contribution and we'll make sure you're credited.
[/fusion_text][/fusion_builder_column][/fusion_builder_row][/fusion_builder_container]
"""
HTML_ROLE_PREFIX = f"""
[fusion_builder_container hundred_percent="no" hundred_percent_height="no" hundred_percent_height_scroll="no" hundred_percent_height_center_content="yes" equal_height_columns="no" hide_on_mobile="small-visibility,medium-visibility,large-visibility" background_position="center center" background_repeat="no-repeat" fade="no" background_parallax="none" enable_mobile="no" parallax_speed="0.3" video_aspect_ratio="16:9" video_loop="yes" video_mute="yes" border_style="solid" admin_toggled="no" type="legacy"][fusion_builder_row][fusion_builder_column type="2_5" layout="1_2" spacing="" center_content="no" link="" target="_self" min_height="" hide_on_mobile="small-visibility,medium-visibility,large-visibility" class="" id="" background_color="" background_image="" background_position="left top" background_repeat="no-repeat" hover_type="none" border_color="" border_style="solid" border_position="all" padding_top="" padding_right="" padding_bottom="" padding_left="" margin_top="" margin_bottom="" animation_type="" animation_direction="left" animation_speed="0.3" animation_offset="" last="false" border_sizes_top="0" border_sizes_bottom="0" border_sizes_left="0" border_sizes_right="0" first="true" spacing_right="" type="2_5"][fusion_title hide_on_mobile="small-visibility,medium-visibility,large-visibility" class="" id="" content_align="left" size="1" font_size="" line_height="" letter_spacing="" margin_top="" margin_bottom="" text_color="" style_type="default" sep_color=""]
<h3 data-fontsize="40" data-lineheight="58">
"""
HTML_ROLE_POSTFIX = f"""
</h3>
[/fusion_title][/fusion_builder_column][fusion_builder_column type="3_5" layout="1_2" spacing="" center_content="no" link="" target="_self" min_height="" hide_on_mobile="small-visibility,medium-visibility,large-visibility" class="" id="" background_color="" background_image="" background_position="left top" background_repeat="no-repeat" hover_type="none" border_color="" border_style="solid" border_position="all" padding_top="" padding_right="" padding_bottom="" padding_left="" margin_top="" margin_bottom="" animation_type="" animation_direction="left" animation_speed="0.3" animation_offset="" last="true" border_sizes_top="0" border_sizes_bottom="0" border_sizes_left="0" border_sizes_right="0" first="false" type="3_5"][fusion_text columns="4" column_min_width="" column_spacing="" rule_style="default" rule_size="" rule_color="" hide_on_mobile="small-visibility,medium-visibility,large-visibility" class="" id=""]
"""
HTML_ROLE_END = f"""
[/fusion_text][/fusion_builder_column][/fusion_builder_row][/fusion_builder_container]
"""
HTML_END = f"""
[fusion_builder_container hundred_percent="no" hundred_percent_height="no" hundred_percent_height_scroll="no" hundred_percent_height_center_content="yes" equal_height_columns="no" hide_on_mobile="small-visibility,medium-visibility,large-visibility" background_position="center center" background_repeat="no-repeat" fade="no" background_parallax="none" enable_mobile="no" parallax_speed="0.3" video_aspect_ratio="16:9" video_loop="yes" video_mute="yes" border_style="solid" admin_toggled="no" type="legacy"][fusion_builder_row][fusion_builder_column type="1_1" layout="1_1" spacing="" center_content="no" link="" target="_self" min_height="" hide_on_mobile="small-visibility,medium-visibility,large-visibility" class="" id="" background_color="" background_image="" background_position="left top" background_repeat="no-repeat" hover_type="none" border_color="" border_style="solid" border_position="all" padding_top="" padding_right="" padding_bottom="" padding_left="" margin_top="" margin_bottom="" animation_type="" animation_direction="left" animation_speed="0.3" animation_offset="" last="true" border_sizes_top="0" border_sizes_bottom="0" border_sizes_left="0" border_sizes_right="0" first="true" type="1_1"][fusion_text columns="" column_min_width="" column_spacing="" rule_style="default" rule_size="" rule_color="" content_alignment_medium="" content_alignment_small="" content_alignment="" hide_on_mobile="small-visibility,medium-visibility,large-visibility" sticky_display="normal,sticky" class="" id="" font_size="" fusion_font_family_text_font="" fusion_font_variant_text_font="" line_height="" letter_spacing="" text_color="" animation_type="" animation_direction="left" animation_speed="0.3" animation_offset=""]
Can't find your name, or want to be credited differently? <a href="https://discord.gg/skywind">Contact us on Discord</a> with details of your contribution and we'll make sure you're credited.
[/fusion_text][/fusion_builder_column][/fusion_builder_row][/fusion_builder_container]
"""

# Assemble output HTML
if DOWNLOAD_AS_HTML:
    OUTPUT_HTML += HTML_START
    for role, credits in OUTPUT_CREDITS:
        OUTPUT_HTML += HTML_ROLE_PREFIX + role + HTML_ROLE_POSTFIX
        OUTPUT_HTML += '\n'.join(credits)
        OUTPUT_HTML += HTML_ROLE_END
    OUTPUT_HTML += HTML_END

    print(f'Downloading {DOWNLOAD_FILENAME}.html')
    open(f'{DOWNLOAD_FILENAME}.html', 'w').write(OUTPUT_HTML)
    files.download(f'{DOWNLOAD_FILENAME}.html')

if DOWNLOAD_AS_JSON or DOWNLOAD_AS_TXT:
    OUTPUT_JSON = json.dumps(OUTPUT_CREDITS, indent=4)

    if DOWNLOAD_AS_JSON:
        print(f'Downloading {DOWNLOAD_FILENAME}.json')
        open(f'{DOWNLOAD_FILENAME}.json', 'w').write(OUTPUT_JSON)
        files.download(f'{DOWNLOAD_FILENAME}.json')
    if DOWNLOAD_AS_TXT:
        print(f'Downloading {DOWNLOAD_FILENAME}.txt')
        open(f'{DOWNLOAD_FILENAME}.txt', 'w').write(OUTPUT_JSON)
        files.download(f'{DOWNLOAD_FILENAME}.txt')
